### 实施 FunkSVD

在此 notebook 中，我们将自己编写实施 FunkSVD 的函数，它会遵守你在上个视频中看到的步骤。如果你认为你还没有准备好自己完成这个任务，可以跳到下个视频，观看我是如何完成这些步骤的。

我们将用之前使用的数据子集检测我们的算法。首先请运行以下单元格。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` 你将使用 **user_movie_subset** 矩阵演示你的 FunkSVD 算法将收敛。在以下单元格中，请参考注释和文档字符串编写你自己的 FunkSVD 函数。你也可以自己完成这些函数，不参考注释部分。你可以随意增减函数代码，使你的代码能正常运行。 

**注意：**此版矩阵分解没有 ∑ 矩阵。

In [3]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0 # sum of squared errors
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` 请在 **user_movie_subset** 数据集上尝试你的函数。先将潜在特征设为 4 个，学习速率设为 0.005，并迭代 10 次。在计算生成的 U 和 V 矩阵的点积时，生成的 **user_movie** 矩阵与原始数据子集有何区别？

In [4]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 43.626020
2 		 14.194749
3 		 3.625608
4 		 2.669425
5 		 2.601501
6 		 2.564697
7 		 2.530203
8 		 2.493908
9 		 2.454667
10 		 2.412149


In [5]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.23662119  8.85714107 10.82659132  9.86997579]
 [ 9.25684029  6.6193294   8.84262651  9.12802047]
 [ 7.79708888  7.33448212  8.43838502  7.40091712]
 [ 9.6361636   7.93495983  9.82330193  9.91542183]
 [ 8.64005437  7.17047691  9.03195638  8.58601546]
 [ 6.70021278  6.14321055  7.11582045  6.94548669]
 [ 9.03882346  8.12897516  9.6859096   9.25072063]
 [ 8.40193385  7.24698665  8.68957727  7.99570172]
 [ 8.20407268  7.43362156  8.71602979  8.81309668]
 [ 7.96678578  6.5760213   8.17079396  7.50712166]
 [ 9.1716272   7.70095496  9.29621362  8.74680998]
 [ 9.62436225  8.48247782  9.95160609  9.59435203]
 [ 9.33425151  8.48506527  9.92757038  8.82599989]
 [ 6.10939452  6.53697902  7.00115327  6.45121096]
 [ 9.91903841  8.01081303 10.14395805 10.00173508]
 [ 9.43717628  8.81168894 10.14800659  9.81770043]
 [ 8.21217544  7.3984771   8.57495184  8.35884748]
 [ 7.2734385   5.72355046  7.20389651  7.48187682]
 [ 8.10541489  7.7384182   8.85209573  8.24960126]
 [ 8.09004224  7.01990086  8.57

**请在此处填写你的结论。**
**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations.  You can see some extreme low values that are not captured well yet.  The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7.  Clearly the model is not done learning, but things are looking good.**

`3.` 再在 **user_movie_subset** 数据集上尝试你的函数。这次潜在特征为 4 个，学习速率设为 0.005。但是，迭代次数提高到 250 次。在计算生成的 U 和 V 矩阵的点积时，生成的 **user_movie** 矩阵与原始数据子集有何区别？迭代 250 次后，误差会怎样？

In [6]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 47.186496
2 		 17.219530
3 		 4.348557
4 		 2.844661
5 		 2.754914
6 		 2.736003
7 		 2.723188
8 		 2.709995
9 		 2.695118
10 		 2.678185
11 		 2.658886
12 		 2.636826
13 		 2.611525
14 		 2.582438
15 		 2.548970
16 		 2.510488
17 		 2.466348
18 		 2.415916
19 		 2.358613
20 		 2.293963
21 		 2.221656
22 		 2.141622
23 		 2.054100
24 		 1.959700
25 		 1.859446
26 		 1.754769
27 		 1.647454
28 		 1.539527
29 		 1.433097
30 		 1.330170
31 		 1.232472
32 		 1.141313
33 		 1.057512
34 		 0.981395
35 		 0.912855
36 		 0.851446
37 		 0.796500
38 		 0.747231
39 		 0.702815
40 		 0.662461
41 		 0.625437
42 		 0.591101
43 		 0.558902
44 		 0.528382
45 		 0.499171
46 		 0.470981
47 		 0.443597
48 		 0.416870
49 		 0.390711
50 		 0.365080
51 		 0.339979
52 		 0.315449
53 		 0.291553
54 		 0.268376
55 		 0.246011
56 		 0.224557
57 		 0.204108
58 		 0.184748
59 		 0.166546
60 		 0.149553
61 		 0.133801
62 		 0.119298
63 		 0.106036
64 		

In [7]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99999702  9.99999856  9.99999945 10.00000281]
 [ 9.99999711  3.99999867  8.99999952 10.00000271]
 [ 8.0000065   9.00000313 10.00000122  4.99999395]
 [ 8.99999796  7.99999905  9.99999964 10.00000191]
 [ 9.99999735  4.99999882  8.99999958  9.00000248]
 [ 6.00001711  4.00000823 10.00000315  5.99998396]
 [ 8.99999375  7.99999686  9.99999872  9.00000584]
 [10.00000914  5.00000429  9.0000016   7.99999146]
 [ 6.99998869  7.99999446  9.99999781  8.00001056]
 [ 8.99997929  4.99999008  8.99999621  7.00001938]
 [ 9.00000981  8.00000492 10.00000203  7.99999087]
 [ 8.999996    9.99999823  9.9999994   9.00000376]
 [10.00000517  9.0000026  10.00000109  7.99999521]
 [ 5.0000011   8.00000065  5.00000032  7.99999899]
 [10.00001448  8.00000696 10.00000269  9.99998648]
 [ 9.          8.99999982  9.99999983  9.99999999]
 [ 8.99999024  7.9999952   7.99999809  8.00000913]
 [10.00000456  8.00000224  1.00000092  9.99999581]
 [ 5.00000212  6.000001   10.00000037  9.99999798]
 [ 7.9999933   6.99999675  9.99

**请在此处填写你的结论。**
**In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error. I obtained 0 MSE on iteration 165.**

上次在此矩阵里放入 **np.nan** 值时，python 中的整个 SVD 算法崩溃了。我们看看使用你的 FunkSVD 函数是否依然会这样。在以下单元格中，我在你的 numpy 数组的第一个单元格中放入了一个 NaN 值。  

`4.` 假设潜在特征有 4 个，学习速率为 0.005，并且迭代 250 次。你能够运行你的 SVD 并且不崩溃吗（内置的 python 方法会崩溃）？你获得了 NaN 值的预测值吗？缺失值的预测是多少？请在以下单元格中回答这些问题。

In [8]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [9]:
# run SVD on the matrix with the missing value
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 52.152034
2 		 23.133246
3 		 6.445888
4 		 3.143671
5 		 2.710574
6 		 2.604873
7 		 2.555339
8 		 2.518622
9 		 2.484467
10 		 2.450205
11 		 2.415171
12 		 2.379282
13 		 2.342630
14 		 2.305363
15 		 2.267621
16 		 2.229500
17 		 2.191022
18 		 2.152108
19 		 2.112562
20 		 2.072059
21 		 2.030142
22 		 1.986232
23 		 1.939644
24 		 1.889618
25 		 1.835360
26 		 1.776091
27 		 1.711121
28 		 1.639929
29 		 1.562250
30 		 1.478171
31 		 1.388210
32 		 1.293364
33 		 1.195107
34 		 1.095317
35 		 0.996129
36 		 0.899730
37 		 0.808128
38 		 0.722936
39 		 0.645234
40 		 0.575513
41 		 0.513720
42 		 0.459375
43 		 0.411721
44 		 0.369877
45 		 0.332955
46 		 0.300144
47 		 0.270756
48 		 0.244234
49 		 0.220150
50 		 0.198183
51 		 0.178094
52 		 0.159710
53 		 0.142898
54 		 0.127555
55 		 0.113592
56 		 0.100930
57 		 0.089490
58 		 0.079196
59 		 0.069969
60 		 0.061729
61 		 0.054397
62 		 0.047894
63 		 0.042141
64 		

In [10]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.108588725589916:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


下面扩展到更真实的例子上。遗憾的是，依然不建议在本地机器上对整个用户-电影矩阵运行此函数。但是，我们能够查看这个示例扩展到 1000 个用户的情况。在上述部分，你使用了一个非常小的数据子集，并且没有缺失值。

`5.` 考虑到此矩阵的大小，运行时间会比较长。假设超参数如下所示：潜在特征为 4 个，学习速率为0.005，迭代次数为20。运行时间会很久，吃点东西，散散步吧。

In [11]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.116482
2 		 10.722545
3 		 7.384865
4 		 5.708581
5 		 4.643374
6 		 3.894543
7 		 3.337359
8 		 2.906582
9 		 2.564096
10 		 2.286160
11 		 2.057181
12 		 1.866413
13 		 1.706107
14 		 1.570440
15 		 1.454894
16 		 1.355878
17 		 1.270498
18 		 1.196401
19 		 1.131667
20 		 1.074731


`6.` 现在你已经获得了每个用户-电影对的预测值，请根据你的结果回答几个问题。请在以下每个变量对应的括号里填写正确的值，并使用以下测试检测你的答案。

In [12]:
# Replace each of the comments below with the correct values
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings 
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [13]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?


```python

```